In [1]:
import os
from dotenv import load_dotenv

import torch
import transformers
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, pipeline
from datasets import load_dataset

import gradio as gr
from IPython.display import display, Markdown
import random

In [2]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

In [3]:
def print_markdown(text):
    """Better output format for notebooks"""
    display(Markdown(text))

In [4]:
print(torch.cuda.is_available())

True


In [11]:
dataset_id = "PaulAdversarial/all_news_finance_sm_1h2023"

print(f"Loading dataset: {dataset_id}")

Loading dataset: PaulAdversarial/all_news_finance_sm_1h2023


In [12]:
news_dataset = load_dataset(dataset_id, split="train")

DatasetNotFoundError: Dataset 'PaulAdversarial/all_news_finance_sm_1h2023' doesn't exist on the Hub or cannot be accessed.

In [7]:
news_dataset

Dataset({
    features: ['instruction', 'output', 'text'],
    num_rows: 5062
})

In [8]:
print(news_dataset.features)

{'instruction': Value('string'), 'output': Value('string'), 'text': Value('string')}


In [9]:
news_dataset.to_pandas()

,instruction,output,text
0,"Dow drops 400 points, turns negative for the y...",Regional banks led the broader market lower as...,Below is an instruction that describes a task....
1,Bitcoin drops with stocks as analyst warns of ...,Bitcoin dips as the U.S. banking crisis engulf...,Below is an instruction that describes a task....
2,Bitcoin Price Analysis: 29370 Tested After Su...,Bitcoin (BTC/USD) sought to add to recent gain...,Below is an instruction that describes a task....
3,"Bitcoin Is 75% To Halving, Here's How Past Cyc...",The current Bitcoin cycle is now 75% on the wa...,Below is an instruction that describes a task....
4,"Iron Mountain FFO of $0.71 beats by $0.03, rev...",Iron Mountain press release (IRM): Q1 FFO of...,Below is an instruction that describes a task....
...,...,...,...
5057,"Ethereum Merge can trigger high volatility, Bi...",BitMEX CEO Alexander Höptner said that their t...,Below is an instruction that describes a task....
5058,EXPLAINER: Is COVID-19 winding down? Scientist...,Is the coronavirus on its way out? You might t...,Below is an instruction that describes a task....
5059,How Bitcoin Educates The World About Finance,Bitcoin brings financial education to the fore...,Below is an instruction that describes a task....
5060,Crypto Hedge Fund Three Arrows Capital Wallet ...,The removal of liquidity was the first activit...,Below is an instruction that describes a task....


In [10]:
def combine_news(example):
    title = example.get("title", "") or ""
    description = example.get("description", "") or ""

    return {"full_text": f"Title: {title}\nDescription: {description}"}